In [ ]:
from keras.models import Model
from keras.layers import Input,Dense,TimeDistributed

In [ ]:
inp = Input(shape=(6,10))
td1 = TimeDistributed(Dense(5))(inp)
model = Model(inputs=inp,outputs=td1)

In [ ]:
import keras
import keras.backend as K
from keras.layers import RNN

class CCell(keras.layers.Layer):
    def __init__(self,units,**kwargs):
        self.units = units
        self.state_size = units
        super(CCell,self).__init__(**kwargs)
        
    def build(self,input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1],self.units),
                                      initializer="uniform",
                                      name='kernel')
        self.rec_kernel = self.add_weight(shape=(self.units,self.units),initializer="uniform",name="rec_kernel")
        self.built =  True
        return
    
    def call(self,inputs,states):
        p_out = states[0]
        output = K.dot(inputs,self.kernel)+K.dot(p_out,self.rec_kernel)
        return output,[output]
    
    
inp = Input(shape=(6,10),name="inp")
td1 = TimeDistributed(Dense(5),name="td")(inp)
rn = RNN(CCell(5),name="rnn")(td1)
model = Model(inputs=inp,outputs=rn)

# inp = Input(shape=(6,10))
# td1 = TimeDistributed(Dense(5,trainable=False,use_bias=False))(inp)
# rn = RNN(CCell(5))(td1)
# model = Model(inputs=inp,outputs=rn)

In [ ]:
from st_global import timer_decorator

In [ ]:
@timer_decorator
def some_fun(func):
    print func
    time.sleep(10)
    return func+19

In [ ]:
some_fun(10)

In [ ]:
import copy
import numpy as np

import st_keras
reload(st_keras)

import st_global
reload(st_global)

from st_global import isinstance_fromlist, hasattr_fromlist,iterate
from st_keras import iterateKerasLayers,NNModel
from st_global import set_path,apply_operation,get_path,exist_path

import networkx as nx

from IPython.display import SVG

def fill_storage(storage,layer,types_to_copy,verbose=1):
    for attr in layer.__dict__:
        if isinstance_fromlist(layer.__dict__[attr],types_to_copy):
            if verbose>0:
                print "copy ",attr," from ",layer
            storage[attr] = layer.__dict__[attr]
    
    if hasattr(layer,"layer"):
        storage["layer"] = {}
        fill_storage(storage["layer"],layer.layer,types_to_copy)
    if hasattr(layer,"cell"):
        storage["cell"] = {}
        fill_storage(storage["cell"],layer.cell,types_to_copy)

    return

def set_keras_layer(storage,layer,types_to_copy,verbose=1):
    for key in storage:
        if not hasattr(layer,key):
            if verbose>0:
                print "-",key," not exists in ",layer
            continue
            
        if key!="layer" and key!="cell":
            if isinstance_fromlist(layer.__dict__[key],types_to_copy):
                K.set_value(layer.__dict__[key],storage[key])
            else:
                if verbose>0:
                    print "-",key," in ",layer," (not in types_to_copy) "        
        else:
            set_keras_layer(storage[key],layer.__dict__[key],types_to_copy)
        
    return


In [ ]:
nn_model = NNModel()
nn_model.fromKeras(model)
nn_model.drawGraph()

In [ ]:
import st_graph
reload(st_graph)
from st_graph import get_leaves, get_roots,add_depth
import st_keras
reload(st_keras)
from st_keras import draw_weights
import st_graphics 
reload(st_graphics)
from st_graphics import bar_fromarray

In [ ]:
inp = Input(shape=(6,10),name="inp")
td1 = TimeDistributed(Dense(5),name="td")(inp)
rn = RNN(CCell(5),name="rnn")(td1)
model2 = Model(inputs=inp,outputs=rn)


nn_model2 = NNModel()
nn_model2.fromKeras(model2)
fig,ax = draw_weights(nn_model2,layers=None,names=["kernel"])

In [ ]:
fig,ax = draw_weights(nn_model,layers=None,names=["kernel"])

In [ ]:
fig,ax = draw_weights(nn_model2-nn_model,layers=None,names=["kernel"])

In [ ]:
inp = Input(shape=(6,10),name="inp")
td1 = TimeDistributed(Dense(5),name="td")(inp)
rn = RNN(CCell(5),name="rnn")(td1)
model4 = Model(inputs=inp,outputs=rn)

nn_model4 = NNModel()
nn_model4.fromKeras(model4)
fig,ax = draw_weights(nn_model4,layers=None,names=["kernel"])

In [ ]:
fig,ax = draw_weights(nn_model4-nn_model,layers=None,names=["kernel"])

In [ ]:
nn_model.setKeras(model4)

In [ ]:
nn_model5 = NNModel()
nn_model5.fromKeras(model4)

In [ ]:
fig,ax = draw_weights(nn_model4-nn_model,layers=None,names=["kernel"])

In [ ]:
fig,ax = draw_weights(nn_model5-nn_model,layers=None,names=["kernel"])

In [ ]:
_path = list(path_list[0])
_path.insert(1,"weights")
print _path
exist_path(nn_model.graph.nodes[_path[0]],_path[1:])
#nn_model.graph.nodes[_path[0]]["weights"]["layer"]["kernel"]

In [ ]:
res = nn_model.graph.subgraph(["input_3"])

In [ ]:
res.graph

In [ ]:
# load values through batch_get_value

res = {}

paths = []
keras_variables = []
for el in iterateKerasLayers(model,types=[K.variable([0]).__class__]):
    paths.append(el[0])
    keras_variables.append(el[1])

# we can replace batch_get_value with cycle with K.eval
keras_values = K.batch_get_value(keras_variables)
    
for path,val in zip(paths,keras_values):
    set_path(res,path[1:],val)

In [ ]:
from keras.layers import Add

inp = Input(shape=(6,10))
td1 = TimeDistributed(Dense(5))(inp)
rn = RNN(CCell(5))(td1)
rn2 = RNN(CCell(5))(td1)
ad = Add()([rn,rn2])
model = Model(inputs=inp,outputs=[rn,rn2,ad])

res = model.get_config()

In [ ]:
import networkx as nx

def makeNXDiGraphfromConfig(config):
    layers = config["layers"]
    
    graph = nx.DiGraph()
    
    for lr in layers:
        graph.add_node(lr["name"])
        
    for lr in layers:
        if len(lr["inbound_nodes"])>0:
            for _node in lr["inbound_nodes"][0]:
                graph.add_edge(_node[0],lr["name"])
            
    return graph

In [ ]:
graph = makeNXDiGraphfromConfig(res)

In [ ]:
import networkx as nx
import pydot
from IPython.display import SVG

def draw_graph(graph):
    graph = nx.drawing.nx_pydot.to_pydot(graph)
    svg_pic = graph.create_svg()
    return SVG(svg_pic)

In [ ]:
import keras.backend as K

In [ ]:
input_data = np.random.rand(100,6,10)
output_data = model.predict(input_data)

In [ ]:
input_params = []
input_params.extend(model.inputs)
output_params = [model.layers[2].output]
fun = K.function(input_params,output_params)
rnn_output = fun([input_data])

In [ ]:
rnn_output[0].shape

In [ ]:
output_data[0].shape

In [ ]:
res = rnn_output[0]-output_data[0]
print np.max(np.abs(res))

In [ ]:
print K.variable([0,0]).__class__
print K.constant([0,0]).__class__
print K.placeholder([0,0]).__class__